In [111]:
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
import requests
from urllib.request import urlopen
from datetime import datetime


In [133]:
# scrpae movies names and their links
url = "https://www.boxofficemojo.com/month/may/2015/?ref_=bo_ml_table_6"
response = requests.get(url)

#page = urlopen(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')

In [134]:
#find movie name from a tag
a_tags = soup.find_all('a')

movies_names = []
movies_links = []
for i in a_tags:
    href = i['href']
    if href.startswith("/release"):
        movies_names.append(i.text)
        movies_links.append("https://www.boxofficemojo.com"+i['href'])
    

In [135]:
print(len(movies_names))
print(len(movies_links))


190
190


In [17]:
# find release date from table cell
#td_date_tags = soup.find_all('td', class_="a-text-left mojo-field-type-date a-nowrap")
#
#movies_release = []
#for i in td_date_tags:
#    datetime_object = datetime.strptime(i.text + " 2015", '%b %d %Y')
#    movies_release.append(datetime_object)

In [137]:
# open each link and find
# 1. release date from table - the first part of date range
# 2. scrape all table
# 3. clean up date 

df_all = pd.DataFrame()

#for i in range(2):
for i in range(len(movies_links)):
    #open page
    #print(i)
    url = movies_links[i]
    response = requests.get(url)
    page = response.text
    print(response.url)
    if (not "week" in response.url): #not weekly or weekend
        soup = BeautifulSoup(page, 'lxml')

        # find release date first part
        a_tags = soup.find_all('a', class_="a-link-normal")
        release_date = None

        for a_tag in a_tags:
            href = a_tag['href']
            if release_date == None and re.match(r'^\/date\/[\d]', href): 
                release_date = datetime.strptime(a_tag.text, '%b %d, %Y')

        # get table
        table = soup.find('table')
        table_rows = table.find_all("tr")
        l = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)

        print(movies_names[i])
        table_df = pd.DataFrame(l, columns=["Date_str", "DOW", "Rank", "Daily", "YD", "LW", "Theatres", "Avg", "ToDate", "Day","Hidden"])
        # add movie name and release date
        table_df["movie_name"] = movies_names[i]
        table_df["release_date"] = release_date


        date_split = table_df["Date_str"].str.split(r", \d\d\d\d*", expand = True)
        date_year = table_df["Date_str"].str.extract(r"(, \d\d\d\d)")
        date_split["year"] = date_year
        date_split["date_str"] = date_split[0] + date_split["year"]
        date_split["date"] = pd.to_datetime(date_split["date_str"], format='%b %d, %Y')

        #add date
        table_df["date"] = date_split["date"]
        table_df = table_df.iloc[1:,]

        df_all = df_all.append(table_df)

#df_all
df_all.to_csv("movies_may_2015_gbo.csv", index=False)


https://www.boxofficemojo.com/release/rl675644929/?ref_=bo_md_table_1
Avengers: Age of Ultron
https://www.boxofficemojo.com/release/rl3932718593/?ref_=bo_md_table_2
Pitch Perfect 2
https://www.boxofficemojo.com/release/rl1851098625/?ref_=bo_md_table_3
Mad Max: Fury Road
https://www.boxofficemojo.com/release/rl2051114497/?ref_=bo_md_table_4
Tomorrowland
https://www.boxofficemojo.com/release/rl1433372161/?ref_=bo_md_table_5
San Andreas
https://www.boxofficemojo.com/release/rl74155521/?ref_=bo_md_table_6
Poltergeist
https://www.boxofficemojo.com/release/rl3917186561/?ref_=bo_md_table_7
Hot Pursuit
https://www.boxofficemojo.com/release/rl1045661185/?ref_=bo_md_table_8
Furious 7
https://www.boxofficemojo.com/release/rl2705360385/?ref_=bo_md_table_9
The Age of Adaline
https://www.boxofficemojo.com/release/rl3345581569/?ref_=bo_md_table_10
Paul Blart: Mall Cop 2
https://www.boxofficemojo.com/release/rl1968604673/?ref_=bo_md_table_11
Home
https://www.boxofficemojo.com/release/rl1481672193/?ref

https://www.boxofficemojo.com/release/rl105678337/?ref_=bo_md_table_100
Dark Star: HR Gigers Welt
https://www.boxofficemojo.com/release/rl2154989057/?ref_=bo_md_table_101
Sunshine Superman
https://www.boxofficemojo.com/release/rl189761025/?ref_=bo_md_table_102
Desert Dancer
https://www.boxofficemojo.com/release/rl2017560065/weekend/
https://www.boxofficemojo.com/release/rl1027769857/?ref_=bo_md_table_104
Animals
https://www.boxofficemojo.com/release/rl591758849/weekend/
https://www.boxofficemojo.com/release/rl726631937/?ref_=bo_md_table_106
Deli Man
https://www.boxofficemojo.com/release/rl423921153/?ref_=bo_md_table_107
She's Beautiful When She's Angry
https://www.boxofficemojo.com/release/rl3866723841/weekend/
https://www.boxofficemojo.com/release/rl3211757057/weekend/
https://www.boxofficemojo.com/release/rl3192358401/?ref_=bo_md_table_110
Blackbird
https://www.boxofficemojo.com/release/rl2353497601/?ref_=bo_md_table_111
Black or White
https://www.boxofficemojo.com/release/rl37469895